In [14]:
import matplotlib as plt 
from tqdm import tqdm
import numpy as np
from MH_coupling import *

In [15]:
def U(x):
    return np.abs(x-2)

In [ ]:
if __name__ == "__main__":
    v = 0.1
    x_init = np.random.randn(1)
    y_init = np.random.randn(1)
    n_iter = 100
    samples = metropolis_sampler(x_init, v, U, n_iter)
    print(samples)


TypeError: float() argument must be a string or a real number, not 'function'

In [10]:
type(samples[0])

numpy.ndarray

In [12]:
type(np.log(6))

numpy.float64